In [11]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 
device = "cuda"

In [12]:
from __future__ import print_function

import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [13]:
import import_ipynb
from Datagen import Data

In [14]:
def likeliness(y_pred, y):
    pred = torch.zeros([len(y_pred), y.shape[1]])
    for idx, unit in enumerate(torch.argmax(y_pred, axis = 1)):
        pred[idx, unit] = 1
    return accuracy_score(pred.cpu(), y.cpu())

In [15]:
class MyDataset(Dataset):
    def __init__(self, x_train, x_test, y_train, y_test, train=True):
        super(MyDataset, self).__init__()
        self.transforms = transforms.ToTensor()
        self.x = x_train if train else x_test
        self.y = y_train if train else y_test
        
    def __getitem__(self, index):
        x = self.x[index, ...]
        y = self.y[index, ...]
        return x, y
    
    def __len__(self):
        return len(self.x)

In [16]:
def load_data():
    seglength = 300
    train_ratio = 0.8
    val_ratio = 0.1

    focal_train, focal_test, focal_train_labels, focal_test_labels = Data('D:\EEG\FOCAL', train_ratio, seglength, class_num = 1)
    ige_train, ige_test, ige_train_labels, ige_test_labels = Data('D:\EEG\IGE', train_ratio, seglength, class_num = 2)
    
    X_train = torch.cat((focal_train, ige_train))
    Y_train = torch.cat((focal_train_labels, ige_train_labels))
    X_test = torch.cat((focal_test, ige_test))
    Y_test = torch.cat((focal_test_labels, ige_test_labels))

    del focal_train, focal_test, focal_train_labels, focal_test_labels, ige_train, ige_test, ige_train_labels, ige_test_labels

    pnes_train, pnes_test, pnes_train_labels, pnes_test_labels = Data('D:\EEG\PNES', train_ratio, seglength, class_num = 3)

    X_train = torch.cat((X_train, pnes_train))
    Y_train = torch.cat((Y_train, pnes_train_labels))
    X_test = torch.cat((X_test, pnes_test))
    Y_test = torch.cat((Y_test, pnes_test_labels))

    del pnes_train, pnes_test, pnes_train_labels, pnes_test_labels

    temporal_train, temporal_test, temporal_train_labels, temporal_test_labels = Data('D:\EEG\TLE', train_ratio, seglength, class_num = 4)

    X_train = torch.cat((X_train, temporal_train))
    Y_train = torch.cat((Y_train, temporal_train_labels))
    X_test = torch.cat((X_test, temporal_test))
    Y_test = torch.cat((Y_test, temporal_test_labels))

    del temporal_train, temporal_test, temporal_train_labels, temporal_test_labels

    X_val = X_train[:int(val_ratio * len(Y_train))]
    Y_val = Y_train[:int(val_ratio * len(Y_train))]
    X_train = X_train[int(val_ratio * len(Y_train)):]
    Y_train = Y_train[int(val_ratio * len(Y_train)):]

    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [17]:
#x_train, y_train, x_val, y_val, x_test, y_test = load_data()

In [18]:
"""
x_train = x_train.view(-1, 1, 18, 300)
x_test = x_test.view(-1, 1, 18, 300)
x_val = x_val.view(-1, 1, 18, 300)
"""

'\nx_train = x_train.view(-1, 1, 18, 300)\nx_test = x_test.view(-1, 1, 18, 300)\nx_val = x_val.view(-1, 1, 18, 300)\n'

In [19]:

x_train = torch.rand([42583, 1, 18, 300])
y_train = torch.ones([42583, 4]) 
y_train[:, :3] = 0
x_val = torch.rand([4731, 1, 18, 300])
y_val = torch.ones([4731, 4]) 
y_val[:, :3] = 0
x_test = torch.rand([11828, 1, 18, 300])
y_test = torch.ones([11828, 4]) 
y_test[:, :3] = 0


In [20]:
class LSTM_CNN_Spatial(nn.Module):
    def __init__(self, num_classes, batch_size, T, C, input_size, hidden_size,
                 num_layers, spatial_num=8, drop_out=0.5):
        super(LSTM_CNN_Spatial, self).__init__()

        self.N = batch_size
        self.T = T
        self.C = C
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.pool = 4
        self.seq_len = self.T // self.input_size
        self.fc_in = spatial_num * self.hidden_size // self.pool
        
        self._lstm = nn.LSTM(self.input_size, self.hidden_size, 
                            self.num_layers, batch_first=True)
        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])
        
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, spatial_num, (self.C, 1)),
            nn.BatchNorm2d(spatial_num),
            nn.ELU(),
            nn.AvgPool2d((1, self.pool)),
            nn.Dropout(drop_out)
        )
        
        self.fc = nn.Linear(self.fc_in , num_classes)
        
    def forward(self, x):
        # input shape of x: (N, 1, C, T)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.seq_len, self.input_size)
        _x = None
        for index, lstm in enumerate(self.lstm):
            lstm_out, _ = lstm(x[:, index, :, :], None)
            tmp = lstm_out[:, -1, :]
            tmp = tmp.unsqueeze(0)
            if _x is None:
                _x = tmp
            else:
                _x = torch.cat((_x, tmp), dim=0)
        
        # (C, N, H) ===> (N, 1, C, H)   H: hidden_size
        x = _x.permute(1, 0, 2).unsqueeze(1)
        x = self.block_1(x)
        
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return probas  

In [21]:
def train(model, criterion, optimizer, data_loader, device, train_num, epochs, x_val, y_val, logged=False):
    for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        batch_size = None
        for index, data in enumerate(data_loader):
            x, y = data
            batch_size = x.shape[0] if index == 0 else batch_size
            x, y = x.to(device), y.to(device)
            
            y_pred = model(x)
            _, pred = torch.max(y_pred, 1)
            
            correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())
            
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += float(loss.item())
            
        batch_num = train_num // batch_size
        _loss = running_loss / (batch_num + 1)
        acc = correct_num / train_num * 100
        if not logged:
            print(f'Epoch {epoch+1}/{epochs}\tTrain loss: {_loss:.4f}')
        print(f"train accuracy: {likeliness(model(x_val[:1000].cuda()), y_val[:1000].cuda())}")
        
        print(model(x_val[0].cuda()))
        print('--------------------------------------------')
    
    
    if not logged:
        print('Finish Training!')

In [22]:
def test(model, criterion, data_loader, device, test_num, log, logged = False):
    running_loss = 0.0
    correct_num = 0
    model.eval()
    batch_size = None
    for index, data in enumerate(data_loader):
        x, y = data
        batch_size = x.shape[0] if index == 0 else batch_size
        x, y = x.to(device), y.to(device)
        
        y_pred = model(x)
        _, pred = torch.max(y_pred, 1)
        if sys.platform == 'linux':
            correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())
        else: 
            correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())
        
        loss = criterion(y_pred, y)
        running_loss += float(loss.item())
    
    batch_num = test_num // batch_size
    _loss = running_loss / (batch_num + 1)
    acc = correct_num / test_num * 100
    print(f'Test loss: {_loss:.4f}\tTest acc: {acc:.2f}%')
    if logged:
        log.append(f'{acc:.2f}\t\n')
        with open('result.txt', 'a') as f:
            f.writelines(log)
    
    return model

In [23]:
def main(epochs, batch_size, input_size, hidden_size, num_layers, spatial_num, drop_out, x_val, y_val, logged=False):
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('DEVICE: ', DEVICE)
    
    group = 1
    sorted_ = True
    # sorted_ = False

    # load data from '.npy' file
    # x_train, x_test, y_train, y_test = load_group_eeg_data(date, group, sorted_=sorted_)
    # x: (N, C, T)  N: trials  C: channels  T: times 
    train_num, test_num = x_train.shape[0], x_test.shape[0]
    
    # make dataset for train and test
    train_data = MyDataset(x_train, x_test, y_train, y_test)
    test_data = MyDataset(x_train, x_test, y_train, y_test, train=False)
    train_loader = DataLoader(train_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # model initiation
    # model = LSTM(num_classes=2, input_size=64, hidden_size=256, num_layers=2)
    
    # model = LSTM_CNN(num_classes=2, channels=x_train.shape[1], input_size=input_size, hidden_size=hidden_size, 
    #                  num_layers=num_layers, spatial_num=spatial_num, drop_out=drop_out)
    
    # model = LSTM_CNN_Half(num_classes=2, batch_size=batch_size, T=x_train.shape[-1],
    #                       C=x_train.shape[-2], input_size=input_size, hidden_size=hidden_size,
    #                       num_layers=num_layers, spatial_num=spatial_num)
    
    model = LSTM_CNN_Spatial(num_classes=4, batch_size=batch_size, T=x_train.shape[-1],
                          C=x_train.shape[-2], input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, spatial_num=spatial_num)
    
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)

    log = []
    if logged:
        log.append(f'{epochs}\t{batch_size}\t{input_size}\t{hidden_size}\t'
                   f'{num_layers}\t{spatial_num}\t{drop_out}\t')
    train(model, criterion, optimizer, train_loader, DEVICE,train_num, epochs, x_val, y_val, logged=False)
    test(model, criterion, test_loader, DEVICE, test_num, log, logged = False)

In [24]:
torch.cuda.empty_cache()

In [25]:
epochs = 200
batch_size = 256
input_size = 30
hidden_size = 16
num_layers = 4
spatial_num = 8
drop_out = 0.5
logged = False
model = main(epochs, batch_size, input_size, hidden_size, num_layers, spatial_num, drop_out, x_val, y_val, logged=False)

DEVICE:  cuda:0


  0%|          | 1/200 [00:04<14:55,  4.50s/it]

Epoch 1/200	Train loss: 0.8331
train accuracy: 1.0
tensor([[0.0032, 0.0031, 0.0029, 0.9909]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
--------------------------------------------


  1%|          | 2/200 [00:07<11:39,  3.53s/it]

Epoch 2/200	Train loss: 0.7491
train accuracy: 1.0
tensor([[1.5458e-03, 2.7785e-03, 5.7308e-04, 9.9510e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
--------------------------------------------


...

Epoch 3/200	Train loss: 0.7460
train accuracy: 1.0
tensor([[0.0316, 0.0118, 0.0101, 0.9465]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
--------------------------------------------


  2%|▏         | 4/200 [00:13<10:12,  3.13s/it]

Epoch 4/200	Train loss: 0.7450
train accuracy: 1.0
tensor([[1.4311e-04, 2.6624e-05, 2.2201e-05, 9.9981e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
--------------------------------------------


  2%|▏         | 4/200 [00:15<12:34,  3.85s/it]


KeyboardInterrupt: 

In [ ]:
model = LSTM_CNN_Spatial(num_classes=4, batch_size=batch_size, T=x_train.shape[-1],
                          C=x_train.shape[-2], input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, spatial_num=spatial_num).cuda()

likeliness(model(x_train[:1000].cuda()), y_val[:1000].cuda())

0.332